**References** : https://github.com/adwait1291/Nike-Style-Product-Description-Generator-With-GPT3

In [ ]:
!pip install datasets

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

In [ ]:
#Load the pre-trained GPT model and tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [ ]:
# adding padding token
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
df = pd.read_csv("/content/sample_data/Week3NikeProductDescriptionsGenerator.csv")
df.head(3)

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
df['Subtitle'].unique()

In [ ]:
df['Subtitle'].shape

In [ ]:
descriptions = df['Product Description']
descriptions.head(3)

In [ ]:
# tokenize product descriptions
def preprocess(descriptions):
  descriptions = descriptions.tolist()
  encodings = tokenizer(descriptions, truncation=True, padding=True)
  # return tokenizer(descriptions, return_tensors="pt", truncation=True, padding=True)
  return Dataset.from_dict(encodings)

In [ ]:
train_dataset = preprocess(descriptions)

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
# set up the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True
)

In [ ]:
# initialize Trainer for fine-tuning model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

In [ ]:
# fine-tune model
trainer.train()

In [ ]:
# function to generate response based on a prompt
def generate_response(prompt,max_length=1500):
  print(f"Prompt: {prompt}")
  inputs = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
  print(f"Input shape: {inputs.shape}")
  # outputs = model.generate(inputs, max_length=max_length, num_return_sequences=1)
  outputs = model.generate(inputs, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2, pad_token_id=tokenizer.eos_token_id)
  generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return generated_text

In [ ]:
# example prompt
prompt = "Nike Air Max 90"
print(generate_response(prompt))

In [ ]:
# lets play arround with your prompt!
prompt = "Nike SB Dunk Low Pro Premium"
print(generate_response(prompt))

In [ ]:
prompt = "Air Jordan 1 Retro High OG, Highly Recommended, Branded, Worth It with the price"
print(generate_response(prompt))

# **Keep Learning!🔥**